In [1]:
using DataFrames

using Plots
using JLD

using Distributions
using StatsBase
using Iterators
using EmpiricalBayes
using StatPlots

using LaTeXStrings

In [34]:
settings = 104
k_max=12
nreps = 20
nreps_total = k_max*nreps
nmethods=4
methods = ["CEB"; "CEBSCOOP"; "G0"; "G"]

4-element Array{String,1}:
 "CEB"     
 "CEBSCOOP"
 "G0"      
 "G"       

In [8]:
true_dists = [MixtureModel([ Normal(-0.3,.2), Normal(0,0.9)],[0.8, 0.2]), EmpiricalBayes.ash_flattop ];

In [19]:
Base.getindex(t::EmpiricalBayes.BradDeconvolveR, ::Int64) = t
maxbias(t::EmpiricalBayes.BradDeconvolveR) = 0.0;
maxbias(t::DonohoCI) = t.max_bias;

In [5]:
marginal_grid = collect(linspace(-6.5,6.5,1001));
prior_grid = collect(linspace(-3,3,121));
marginal_h = marginal_grid[2]-marginal_grid[1];

In [26]:
d_true1 = NormalConvolutionProblem(true_dists[1], marginal_grid)
d_true2 = NormalConvolutionProblem(true_dists[2], marginal_grid)


EmpiricalBayes.NormalConvolutionProblem(MixtureModel{Distributions.Normal{Float64}}(K = 7)
components[1] (prior = 0.1429): Distributions.Normal{Float64}(μ=-1.5, σ=0.5)
components[2] (prior = 0.1429): Distributions.Normal{Float64}(μ=-1.0, σ=0.5)
components[3] (prior = 0.1429): Distributions.Normal{Float64}(μ=-0.5, σ=0.5)
components[4] (prior = 0.1429): Distributions.Normal{Float64}(μ=0.0, σ=0.5)
components[5] (prior = 0.1429): Distributions.Normal{Float64}(μ=0.5, σ=0.5)
components[6] (prior = 0.1429): Distributions.Normal{Float64}(μ=1.0, σ=0.5)
components[7] (prior = 0.1429): Distributions.Normal{Float64}(μ=1.5, σ=0.5)
, [3.41752e-8, 3.60217e-8, 3.7963e-8, 4.00037e-8, 4.21485e-8, 4.44025e-8, 4.67709e-8, 4.92592e-8, 5.18731e-8, 5.46185e-8  …  5.46185e-8, 5.18731e-8, 4.92592e-8, 4.67709e-8, 4.44025e-8, 4.21485e-8, 4.00037e-8, 3.7963e-8, 3.60217e-8, 3.41752e-8], [-6.5, -6.487, -6.474, -6.461, -6.448, -6.435, -6.422, -6.409, -6.396, -6.383  …  6.383, 6.396, 6.409, 6.422, 6.435, 6.448, 6.461

In [35]:
res_df = DataFrame(true_dist = Distribution[],
    truetheta=Float64[], 
    x=Float64[], m=Int64[],
    σ=Float64[], 
    bias=Float64[], 
    coverage=Float64[], 
    width=Float64[],
    se=Float64[],
    maxbias=Float64[],
    lowerband=Float64[],
    upperband=Float64[],
    method = String[]
)

for comb=1:settings
    @show comb
    for nmethod=1:2
    @show nmethod
    cnt = one(Int)

    point_est = Vector{Float64}(nreps_total)
    bias_calib = Vector{Float64}(nreps_total)
    coverage_calib = Vector{Bool}(nreps_total)
    width_calib = Vector{Float64}(nreps_total)
    maxbias_calib = Vector{Float64}(nreps_total)
    se_calib = Vector{Float64}(nreps_total)

    lower_band_vec = Vector{Float64}(nreps_total)
    upper_band_vec = Vector{Float64}(nreps_total)

    x=0.0
    m=10
    σ=1.0
    true_θ = 0.0 
    true_dist = Normal(0,1)
    d_true = d_true1
        
    method_name = methods[nmethod]

    for sim_batch=1:k_max
    
        sim = load("May22/May22/mysim_$(comb)_$(sim_batch).jld")["res"]
    
            
        x = sim[2][:x]
        m = sim[2][:m]
        σ = sim[2][:σ]

        true_dist = sim[2][:dist]
            
        if Symbol(true_dist) == Symbol(true_dists[1])
            d_true = d_true1
        else
            d_true = d_true2
        end
            
        sim = sim[3]

        target = PosteriorTarget(LFSRNumerator(x))

        true_num, true_denom, true_θ = posterior_stats(d_true, target)

  
        for k=1:nreps
            donoho_res = sim[k][nmethod][1]
            l,r = confint(donoho_res, target) 
            point_est[cnt] = estimate(donoho_res, target)
            bias_calib[cnt] = estimate(donoho_res, target) - true_θ
            coverage_calib[cnt] = r >= true_θ >= l
            width_calib[cnt] = r-l
            maxbias_calib[cnt] = maxbias(donoho_res)
                
            lower_band_vec[cnt] = l
            upper_band_vec[cnt] = r
            cnt +=1
        end
    end

    bias_calib = mean(bias_calib)
    coverage_calib = mean(coverage_calib)
    width_calib = mean(width_calib)
    se_res = std(point_est)
    maxb = mean(maxbias_calib)
    lower_band = mean(lower_band_vec)
    upper_band = mean(upper_band_vec)
        
    push!(res_df, (true_dist, true_θ, x, m, σ, bias_calib, coverage_calib, width_calib, se_res, 
                 maxb, lower_band, upper_band, method_name))
    end
end

#res_df[:σ] = string.(res_df[:σ]);
head(res_df)

comb = 1
nmethod = 1
nmethod = 2
comb = 2
nmethod = 1
nmethod = 2
comb = 3
nmethod = 1
nmethod = 2
comb = 4
nmethod = 1
nmethod = 2
comb = 5
nmethod = 1
nmethod = 2
comb = 6
nmethod = 1
nmethod = 2
comb = 7
nmethod = 1
nmethod = 2
comb = 8
nmethod = 1
nmethod = 2
comb = 9
nmethod = 1
nmethod = 2
comb = 10
nmethod = 1
nmethod = 2
comb = 11
nmethod = 1
nmethod = 2
comb = 12
nmethod = 1
nmethod = 2
comb = 13
nmethod = 1
nmethod = 2
comb = 14
nmethod = 1
nmethod = 2
comb = 15
nmethod = 1
nmethod = 2
comb = 16
nmethod = 1
nmethod = 2
comb = 17
nmethod = 1
nmethod = 2
comb = 18
nmethod = 1
nmethod = 2
comb = 19
nmethod = 1
nmethod = 2
comb = 20
nmethod = 1
nmethod = 2
comb = 21
nmethod = 1
nmethod = 2
comb = 22
nmethod = 1
nmethod = 2
comb = 23
nmethod = 1
nmethod = 2
comb = 24
nmethod = 1
nmethod = 2
comb = 25
nmethod = 1
nmethod = 2
comb = 26
nmethod = 1
nmethod = 2
comb = 27
nmethod = 1


LoadError: [91mFile May22/May22/mysim_27_1.jld cannot be found[39m

In [36]:
save("res_df.jld", "res_df", res_df)